# Alphalensを使って、アルファを評価する

このNotebookは、"TUTORIAL3 Alphalens"の[Lesson2](https://www.quantopian.com/tutorials/alphalens#lesson2)と、[Lesson3](https://www.quantopian.com/tutorials/alphalens#lesson3)をベースに、今回の勉強会に合わせて加筆・修正をしています。

## はじめに
先程のNotebookで、[quant workflow](https://blog.quantopian.com/a-professional-quant-equity-workflow/) を説明しました。このNotebookでは、ワークフロー中の「Alpha discovery」を実際に試してみます。

アルファの発見、言い換えれば、「将来のリターンを生み出すファクター」を見つけ出すためには

* 仮説（アルファの候補）を立て
* 仮説が正しいかどうかを検証する
* うまく行かなければ仮説の立て直し

の繰り返しです。では、仮説の検証はどのように行えばよいでしょうか？また、複数のアルファの候補の中での優劣はどのように決めればよいでしょうか？

Quantopianでは、アルファの発見というワークフローを手助けしてくれるAlphalensというパッケージを提供しています。このNotebookでは、Alphalensを使ったアルファ発見の手順を学習します。


### ファンダメンタルデータを使って、アルファを構築する

いよいよ今回の勉強会のテーマである「ファンダメンタルファクターモデル」の核心に触れていきます。ここでは、「純利益(net income)」と、「時価総額(market cap)」をベースにアルファを構築します。


この表内の２つの変数間の相関を計算したものがICです。アルファの順位が高いとき、将来リターンの順位も高ければ、アルファが将来リターンの予測の良い指標になっている、ということです。



# アルファの良し悪しを計測する指標: IC (Information Coeeficient)

ここでアルファの良し悪しを計測する指標として、IC(Information Coefficient)を導入します。


$$ IC = \rho (\alpha, acutual return) $$

数学的には[スピアマンの順位相関係数](https://ja.wikipedia.org/wiki/%E3%82%B9%E3%83%94%E3%82%A2%E3%83%9E%E3%83%B3%E3%81%AE%E9%A0%86%E4%BD%8D%E7%9B%B8%E9%96%A2%E4%BF%82%E6%95%B0)
という統計学上のテクニックを応用します。

簡単にいうと、**アルファの値（予測リターン）と、将来リターンとの（順位）相関係数を表していて、-1から+1の値を取ります。** 相関係数なので、ICが0以上（※）であれば、そのアルファは何らかの予測能力を持つと考えられます。

（※）ICの値がマイナスの場合は**逆神様**と呼ばれることになるでしょう。




## 【参考】順位相関の考え方

例えば今日（2018/12/15時点）においては、2018/12/14までのデータを利用することができます。
* アルファの順位
  * 2018/12/03時点で利用可能なデータを使って、ユニバース内の銘柄に対してアルファを計算し、アルファの大きい順にソートして順位付けしたもの
* 将来リターンの順位
  * 2018/12/03の株価と、2018/12/10の株価を使って（週次）リターンを計算し、リターンの大きい順位ノートして順位付けしたもの

その結果は、以下のような表になるはずです。

|銘柄|アルファの順位|将来リターンの順位|
|:--:|:-:|:-:|
|A|6|10|
|B|4|1|
|C|5|4|
|D|10|9|
|E|2|3|

もしアルファが将来リターンの予測に役立つのであれば、「アルファの順位が高い⇒将来リターンの順位も高い」という正の相関関係があるはず、というのがICの考え方です。



## Alphalens を使えば全て解決！

ICの計算式を特に記述しませんでしたが、心配しなくて大丈夫です。Quantopianが提供している"Alphalens"というパッケージを使えば、ICの計算や分析結果の可視化を全て自動化してくれます。自分でやらなければならないのは、

* アルファ（ファクター）の構築
* Alphalensで必要なデータの準備

だけです。以下で具体的なコードを見ていきます。

### 下準備：必要なパッケージのインポート
アルファ（ファクター）の構築に必要なパッケージと、Alphalensの利用に必要なパッケージのインポートを行います。
Alphalens 関連のパッケージ以外は、ここまでのNotebookで扱ってきたものです。

In [ ]:
# この例ではfactsetが提供しているデータ・セットを利用
from quantopian.pipeline.data import factset

# Notebook上でパイプラインを実行する上で必要なパッケージ
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline

# ファクター 
from quantopian.pipeline.factors import SimpleMovingAverage

# ユニバース
from quantopian.pipeline.filters import QTradableStocksUS

# Alphalens
from alphalens.tears import create_information_tear_sheet
from alphalens.utils import get_clean_factor_and_forward_returns


### （その１）アルファ（ファクター）の構築

make_pipeline()でアルファの構築を行います。

In [ ]:
def make_pipeline():
    '''
    アルファの構築を行う
    '''
    
    # 過去１年間の純利益の単純移動平均（ファクターの元データ）
    net_income_moving_average = SimpleMovingAverage( 
        inputs=[factset.Fundamentals.net_inc_af], 
        window_length=252
    )
    
    # 過去１年間の時価総額の単純移動平均（ファクターの元データ）
    market_cap_moving_average = SimpleMovingAverage( 
        inputs=[factset.Fundamentals.mkt_val], 
        window_length=252
    )
    
    # 純利益当たりの時価総額
    average_market_cap_per_net_income = (market_cap_moving_average / net_income_moving_average)
    
    # 直近四半期の純利益
    net_income = factset.Fundamentals.net_inc_qf.latest 
    
    # （時価総額移動平均 / 純利益移動平均） *  直近四半期の純利益　⇒ （過去１年間の平均的な）時価総額
    projected_market_cap = average_market_cap_per_net_income * net_income
    
    return Pipeline(
        columns = {'projected_market_cap': projected_market_cap},
        screen = QTradableStocksUS() & projected_market_cap.notnull()
    )

### （その２）Alphalenseで必要なデータの準備

Alphalensで必要なデータの準備は、

* run_pipeline() : アルファの計算を実行
* get_pricing() : パイプラインに含まれる銘柄の株価を取得
* get_clean_factor_and_forward_returns : アルファと株価の紐づけをして、（アルファ計算時点から見て）将来のリターンを行う

をそれぞれ実行すればＯＫです。
そして最後に、

* create_information_tear_sheet で分析結果を可視化

です。この部分でユーザが決めなければいけないのは分析期間だけです。

In [ ]:
# Alphalens で分析させる期間
start_date= '2016-1-1'
end_date='2017-12-1'

# パイプライン実行
pipeline_output = run_pipeline(make_pipeline(), start_date, end_date)

# Alphalenseで必要なデータの準備と実行（この３行を変える必要は多分殆どない）
pricing_data = get_pricing(pipeline_output.index.levels[1], start_date, end_date, fields='open_price')
factor_data = get_clean_factor_and_forward_returns(pipeline_output, pricing_data)
create_information_tear_sheet(factor_data)

### Quantopianのココがすごい！

**アルファの構築以外については全てQuantopianがやってくれます。ユーザはアルファの試行錯誤に専念できるのがQuantopianのスゴいところだと思います** 

# 複数のアルファファクターを合成する

上で実行したアルファは時価総額をベースに計算しました。下の例では、これに"price_to_book"というファクターを**合成して**新しいアルファを構築しようとしています。

In [ ]:
def make_pipeline():

    net_income_moving_average = SimpleMovingAverage( # 1 year moving average of year over year net income
        inputs=[factset.Fundamentals.net_inc_af], 
        window_length=252
    )
    
    market_cap_moving_average = SimpleMovingAverage( # 1 year moving average of market cap
        inputs=[factset.Fundamentals.mkt_val], 
        window_length=252
    )
    
    # １つ目のファクターは先ほどと同じく計算
    average_market_cap_per_net_income = (market_cap_moving_average / net_income_moving_average)
    net_income = factset.Fundamentals.net_inc_qf.latest # the last quarter's net income
    projected_market_cap = average_market_cap_per_net_income * net_income
    
    # ２つ目のファクター登場
    price_to_book = factset.Fundamentals.pbk_qf.latest
    
    # ２つのファクターを合成（ここに注目！）。zscoreってなんぞ？
    factor_to_analyze = projected_market_cap.zscore() + price_to_book.zscore()
    
    return Pipeline(
        columns = {'factor_to_analyze': factor_to_analyze},
        screen = QTradableStocksUS() & factor_to_analyze.notnull()
    )

pipeline_output = run_pipeline(make_pipeline(), start_date, end_date)
pricing_data = get_pricing(pipeline_output.index.levels[1],  start_date, end_date, fields='open_price')
new_factor_data = get_clean_factor_and_forward_returns(pipeline_output, pricing_data)

create_information_tear_sheet(new_factor_data)

# 【ポイント】アルファの合成はzscoreで正規化してから！

* factor_to_analyze = projected_market_cap.zscore() + price_to_book.zscore()

で合成しています。zscore()という新しい関数が登場しています。これは非常に重要なツールです！

アルファが表現している数値には**銘柄間の数値の大小関係（順位）に意味があり、数値の値そのものには意味が殆どありません**。
しかし、複数のアルファファクターを合成する場合には、それぞれのアルファの値の相対的な規模の違いが意味を持ってしまいます。

例えば、平均的なアルファの値が 1～5くらいの値を取るファクターと、平均的なアルファの値が1,000～10,000くらいの値を取るファクターを単純に足し合わせて合成したらどうなるか？を考えれば、その理由がわかると思います。

そこで、複数のアルファを合成する際には、zscoreという統計手法を使ってアルファの値を調整します。
zscoreを使ってアルファの値を調整すると、どのようなマグニチュードのアルファであっても、平均を0として、-5から+5 くらいの範囲内に置き換えてくれます。

これによって、先ほど例に挙げたようなマグニチュードの異なるアルファを合成しても同じような重要度で合成することができます。

**複数のアルファを合成するときに重みづけしたいような場合には、zscore化してから重みづけをすればＯＫ**です。例えば、

* factor_to_analyze = (0.75 * projected_market_cap.zscore()) + (0.25 * price_to_book.zscore())

とすればＯＫです。
元のアルファと比べて予測精度を向上させたかどうかをICでチェックしてみましょう。



# アルファが利益を上げるか？

アルファの予測可能性について、**IC**という指標を用いて検証する方法をみてきました。**ICを比較することで、複数のアルファの候補の中での優劣をつけることが可能になりました。**

複数の候補から選びだしアルファを使って実際に運用を行ったときに利益を上げることができるか？というのが次の興味の対象となります。これはAlgorithmにおけるバックテストでも可能ですが、Notebook内で素早く検証することも可能です。

`create_returns_tear_sheet()` に、アルファ分析データを渡すことで、仮想的なポートフォリオの運用シミュレーション結果をレポートしてくれます。

仮想的なポートフォリオは、ユニバース銘柄を、アルファの値に応じてソートして、ユニバースを５分割することによって作成されます。これをファイナンスの世界では**分位ポートフォリオ（Quantile Portfolio）**と呼びます。

もし、アルファの予測可能性が優秀であれば、

* ５つのポートフォリオの平均的なリターンは、 第１＞第２＞第３＞第４＞第５　というように綺麗に並ぶ
* 第１分位ポートフォリオの累積リターンと、第５分位ポートフォリオの累積リターンの差が綺麗に広がる

でしょう。create_returns_tear_sheet()は、これを可視化してくれる便利なレポート機能です。

## Execise
create_returns_tear_sheetを実行してみて、分位ポートフォリオのリターンを確認し、評価対象となったアルファが適切なものかどうかを判断しましょう。

In [ ]:
from alphalens.tears import create_returns_tear_sheet

create_returns_tear_sheet(factor_data)